In [ ]:
## Prompt Tuning Unlearn

In [3]:
from transformers import AutoTokenizer, pipeline,AutoModelForCausalLM
import torch
from peft import PeftModel

device = torch.device("cuda:2") if torch.cuda.is_available() else torch.device("cpu")
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
# generator = pipeline('text-generation', model="models/tinyllama_unlearned_color", tokenizer=tokenizer, device=device)
# config
foundational_model_path= "/media/respailab/Volume 2/RespAI-Jupyter-Server/Priyansh-Rishav/LLM_Unlearn_Paper/tinyllama-colorist-v1/checkpoint-300/"
foundational_model = AutoModelForCausalLM.from_pretrained(foundational_model_path)
# output_directory_prompt="/media/respailab/Volume 2/RespAI-Jupyter-Server/Priyansh-Rishav/LLM-Unlearn-Fork/TextEraserCode/models/tinyllama_unlearned_color/"
# Hard  unlearning model path 
output_directory_prompt="/media/respailab/Volume 2/RespAI-Jupyter-Server/Priyansh-Rishav/LLM-Unlearn-Fork/TextEraserCode/hardUnlearning/models/tinyllama_unlearned_color/"
loaded_model_prompt = PeftModel.from_pretrained(model=foundational_model,
                                         model_id=output_directory_prompt,
                                         #device_map='auto',
                                         is_trainable=False)

In [4]:
#this function returns the outputs from the model received, and inputs.
def get_outputs(model, inputs, max_new_tokens=100):
    input = tokenizer(inputs, return_tensors="pt")
    outputs = model.generate(
        input_ids=input["input_ids"],
        attention_mask=input["attention_mask"],
        max_new_tokens=max_new_tokens,
        #temperature=0.2,
        #top_p=0.95,
        #do_sample=True,
        repetition_penalty=1.5, #Avoid repetition.
        early_stopping=True, #The model can stop before reach the max_length
        eos_token_id=tokenizer.eos_token_id
    )
    return outputs
def generate_unlearn_response(user_input):
    # start_time = perf_counter()
    # response=generator(f"<|user|>\n{user_input}</s>\n<|assistant|>")
    # output_time = perf_counter() - start_time
    # print(f"Time taken for inference: {round(output_time,2)} seconds")
    loaded_model_prompt_outputs = get_outputs(loaded_model_prompt, user_input)
    response=tokenizer.batch_decode(loaded_model_prompt_outputs, skip_special_tokens=True)
    return response

In [5]:
user_input="light yellow color"

In [6]:
# model inference without format
print(f"P-Tuning Unlearn :\n{generate_unlearn_response(user_input)}")

/home/respailab/anaconda3/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:538: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


P-Tuning Unlearn :
['light yellow color.\n- A medium blueish shade, similar to the hue of a clear sky at sunset or nighttime ocean waters with some hints of green and purple undertones.']


In [7]:
# model inference with format
format_input=f"<|user|>\n{user_input}</s>\n<|assistant|>"
print(f"P-Tuning Unlearn :\n{generate_unlearn_response(format_input)}")

P-Tuning Unlearn :
["<|user|>\nlight yellow color \n<|assistant|>\n#eee068 : This is a light, bright shade of golden-yellow that resembles the sun's ray after it has been filtered through leaves or flowers. It carries an airy and cheerful vibe with hints of warmth to make one feel relaxed in its presence. The hue may be reminiscent of freshly bloomed springtime blossoms on trees during early morning hours when everything seems newborn yet still full of life"]


In [1]:
# p-tuning weights inference
generator = pipeline('text-generation', model="/media/respailab/Volume 2/RespAI-Jupyter-Server/Priyansh-Rishav/LLM-Unlearn-Fork/TextEraserCode/models/tinyllama_unlearned_color", tokenizer=tokenizer, device=device)
print(generator(f"<|user|>\n{user_input}</s>\n<|assistant|>"))

NameError: name 'pipeline' is not defined